# 1.0 Working on Signature Methods

In [1]:
from clustergrammer_widget import *
net = Network(clustergrammer_widget)
df = {}

import clustergrammer_groupby as cby
import gene_exp_10x

In [2]:
### signature distance calculation requirements
from sklearn.metrics import pairwise_distances
from scipy.stats import ttest_ind 

import pandas as pd
import numpy as np
from copy import deepcopy

import matplotlib.pyplot as plt
%matplotlib inline 

### Load Default Dataset

In [3]:
net.load_file('../data/rc_two_cats.txt')
df['ini'] = net.export_df()
df['ini'].shape

(38, 29)

In [4]:
net.cluster()
net.widget()

### Calculate Signatures of Categories

In [5]:
df['ini_mi'] = cby.row_tuple_to_multiindex(df['ini'].transpose())

In [7]:
df['ini_mi'].head()

,,,"(Gene: CDK4, Gene Type: Interesting)","(Gene: LMTK3, Gene Type: Not Interesting)","(Gene: LRRK2, Gene Type: Not Interesting)","(Gene: UHMK1, Gene Type: Not Interesting)","(Gene: EGFR, Gene Type: Interesting)","(Gene: STK32A, Gene Type: Interesting)","(Gene: NRK, Gene Type: Interesting)","(Gene: ERBB2, Gene Type: Not Interesting)","(Gene: ERBB4, Gene Type: Not Interesting)","(Gene: AAK1, Gene Type: Not Interesting)",...,"(Gene: ROS1, Gene Type: Interesting)","(Gene: MAP2K4, Gene Type: Interesting)","(Gene: SRC, Gene Type: Interesting)","(Gene: TGFBR1, Gene Type: Interesting)","(Gene: CAMK2B, Gene Type: Not Interesting)","(Gene: STK24, Gene Type: Interesting)","(Gene: DCLK3, Gene Type: Not Interesting)","(Gene: LATS1, Gene Type: Not Interesting)","(Gene: NEK9, Gene Type: Not Interesting)","(Gene: MYLK3, Gene Type: Not Interesting)"
Cell Line,Category,Gender,,,,,,,,,,,,,,,,,,,,,
H1650,two,Male,-0.792804,0.177621,-0.697876,0.850547,1.412416,-0.388040,1.408537,0.906642,-0.452907,3.579052,...,-0.312364,0.119311,-0.294264,-0.000864,-0.276737,-0.318076,-0.670178,-0.695253,-0.337849,-0.368173
H23,two,Male,0.527687,-0.016061,-0.555610,-0.263280,0.018988,-0.592627,-0.017369,-0.684771,-0.392791,0.923308,...,0.701257,0.593671,-0.618072,0.735638,-0.426081,-0.814111,3.224534,4.299877,-0.535266,0.209192
CAL-12T,two,Male,0.000623,5.422114,-0.360498,0.179253,0.902252,-0.244137,-0.367128,0.015261,-0.374174,-0.651094,...,0.475208,0.489153,-0.252534,-0.680290,-0.160160,0.646545,0.145510,-0.175587,0.803160,0.266318
H358,one,Male,0.356723,1.307040,-0.460237,0.398647,-0.178137,0.740365,0.313254,0.160568,-0.527418,0.952744,...,-0.585297,0.841683,-0.786607,0.040926,-0.890033,0.268372,0.107432,-0.061022,0.275911,-0.100657
H1975,two,Female,0.933286,0.355815,-0.680761,1.537664,0.781819,3.023348,-0.162887,0.365002,-0.320103,-0.212733,...,-0.122694,1.064674,-0.228027,0.359330,-0.437405,-9.425121,-1.120493,-0.391646,0.981343,-0.336792


### Collect Top diff genes using T-test

['five', 'four', 'one', 'three', 'two']

In [41]:
num_top_genes = False
pval_cutoff = 0.05

cell_types = sorted(list(set(df['ini_mi'].index.get_level_values('Category').tolist())))

inst_level = 'Category'

keep_genes = []
keep_genes_dict = {}

for inst_ct in cell_types:
    
    inst_ct_mat = df['ini_mi'].xs(key=inst_ct, level=inst_level)
    inst_other_mat = df['ini_mi'].drop(inst_ct, level=inst_level)
    
    inst_stats, inst_pvals = ttest_ind(inst_ct_mat, inst_other_mat, axis=0)
    
    ser_pval = pd.Series(data=inst_pvals, index=df['ini_mi'].columns.tolist()).sort_values()

    if num_top_genes == False:
        ser_pval_keep = ser_pval[ser_pval < pval_cutoff]

    print(ser_pval_keep)
    inst_keep = ser_pval_keep.index.tolist()
    keep_genes.extend(inst_keep)
    keep_genes_dict[inst_ct] = inst_keep

(Gene: NEK9, Gene Type: Not Interesting)    0.000724
(Gene: ULK4, Gene Type: Interesting)        0.002321
(Gene: ROS1, Gene Type: Interesting)        0.010923
(Gene: MAPK11, Gene Type: Interesting)      0.013899
dtype: float64
(Gene: MAPK4, Gene Type: Interesting)         0.008790
(Gene: TGFBR1, Gene Type: Interesting)        0.010232
(Gene: SRPK3, Gene Type: Not Interesting)     0.013685
(Gene: UHMK1, Gene Type: Not Interesting)     0.016737
(Gene: CAMK2B, Gene Type: Not Interesting)    0.025516
(Gene: GRK1, Gene Type: Not Interesting)      0.036758
dtype: float64
(Gene: STK32A, Gene Type: Interesting)     0.000172
(Gene: STK31, Gene Type: Interesting)      0.002281
(Gene: KDR, Gene Type: Not Interesting)    0.006222
(Gene: NRK, Gene Type: Interesting)        0.011511
(Gene: CDK4, Gene Type: Interesting)       0.045237
dtype: float64
(Gene: LRRK2, Gene Type: Not Interesting)    0.002408
dtype: float64
(Gene: PRKCE, Gene Type: Not Interesting)    0.005459
(Gene: STK39, Gene Type: Inter

In [38]:
ser_pval

(Gene: PRKCE, Gene Type: Not Interesting)     0.005459
(Gene: STK39, Gene Type: Interesting)         0.008790
(Gene: STK24, Gene Type: Interesting)         0.049223
(Gene: LMTK3, Gene Type: Not Interesting)     0.070541
(Gene: INSRR, Gene Type: Not Interesting)     0.077243
(Gene: LATS1, Gene Type: Not Interesting)     0.095774
(Gene: AAK1, Gene Type: Not Interesting)      0.113099
(Gene: PDK4, Gene Type: Not Interesting)      0.208975
(Gene: PRKG2, Gene Type: Not Interesting)     0.228264
(Gene: STK31, Gene Type: Interesting)         0.233379
(Gene: TBK1, Gene Type: Not Interesting)      0.242821
(Gene: DCLK3, Gene Type: Not Interesting)     0.283216
(Gene: KDR, Gene Type: Not Interesting)       0.290763
(Gene: LRRK2, Gene Type: Not Interesting)     0.341282
(Gene: PAK3, Gene Type: Not Interesting)      0.460001
(Gene: ERBB4, Gene Type: Not Interesting)     0.466323
(Gene: MAPK4, Gene Type: Interesting)         0.500857
(Gene: PDGFRA, Gene Type: Interesting)        0.533953
(Gene: MYL

In [35]:
ser_pval[ser_pval < 0.05]

(Gene: PRKCE, Gene Type: Not Interesting)    0.005459
(Gene: STK39, Gene Type: Interesting)        0.008790
(Gene: STK24, Gene Type: Interesting)        0.049223
dtype: float64

In [31]:
inst_stats

array([-0.57817604,  1.88282611, -0.96872281,  0.60111829,  0.59527214,
       -0.62489808, -0.45959691, -0.52487892, -0.7389341 ,  1.63764772,
       -0.52271428, -2.82468387,  0.32493771, -1.19410989,  1.83699349,
        0.54517167, -1.07756185, -0.29008501, -0.74956016, -0.63005868,
        1.28712957,  0.0938872 ,  3.02088963, -1.23283495,  0.6823073 ,
       -0.58610431, -1.21902214,  0.2041413 , -0.52075566,  0.486809  ,
       -0.50915178, -0.57554108,  0.00878442, -2.05935181,  1.09493689,
        1.72600935,  0.59929785, -0.6249342 ])

In [28]:
keep_genes_dict

{'five': [('Gene: NEK9', 'Gene Type: Not Interesting'),
  ('Gene: ULK4', 'Gene Type: Interesting'),
  ('Gene: ROS1', 'Gene Type: Interesting')],
 'four': [('Gene: MAPK4', 'Gene Type: Interesting'),
  ('Gene: TGFBR1', 'Gene Type: Interesting'),
  ('Gene: SRPK3', 'Gene Type: Not Interesting')],
 'one': [('Gene: STK32A', 'Gene Type: Interesting'),
  ('Gene: STK31', 'Gene Type: Interesting'),
  ('Gene: KDR', 'Gene Type: Not Interesting')],
 'three': [('Gene: LRRK2', 'Gene Type: Not Interesting'),
  ('Gene: SRC', 'Gene Type: Interesting'),
  ('Gene: PAK3', 'Gene Type: Not Interesting')],
 'two': [('Gene: PRKCE', 'Gene Type: Not Interesting'),
  ('Gene: STK39', 'Gene Type: Interesting'),
  ('Gene: STK24', 'Gene Type: Interesting')]}